Cartar de posible desvio para la union

In [1]:
import sys
sys.path.append('..')

import pandas as pd
import collections
import os

from docxtpl import DocxTemplate

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [2]:
RUTA_COMPLETA

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [3]:
path_xlsx_avance = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE DE COSECHA V2.xlsx'

In [4]:
xlsx_entregas = pd.read_excel(path_xlsx_avance, sheet_name='DETALLE ENTREGAS')
xlsx_avance = pd.read_excel(path_xlsx_avance, sheet_name = 'AVANCE')

C:\Users\Usuario\anaconda3\envs\utea\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\Usuario\anaconda3\envs\utea\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [5]:
xlsx_entregas = xlsx_entregas.drop(xlsx_entregas.index[-1])
xlsx_entregas['cod_cos'] = xlsx_entregas['cod_cos'].astype(int)
xlsx_entregas['cod_ca'] = xlsx_entregas['cod_ca'].astype(int)
xlsx_entregas['%ENTREGA'] = xlsx_entregas['%ENTREGA'] * 100
xlsx_entregas.head(3)

,inst,cod_cos,cod_ca,nom_ca,TOTAL COMPROMISO,TOTAL ENTREGAS,%ENTREGA
0,1,49,660,AGUILERA ANTELO ALEX ENRIQUE,982.61,627.21,63.831021
1,1,54,3361,ENRIQUEZ ROJAS ALBERTO,1615.39,246.99,15.289806
2,1,55,3460,ESPINOZA MENDOZA SANTIAGO,5288.56,4055.89,76.691765


In [6]:
xlsx_avance.head(3)

,COD COS,COD CA,NOMBRE CAÑERO,COMPROMISO,ENTREGAS,%ENTREGA,AREA,COSECHADO,SIN COSECHAR,SEMILLA,...,TCH EST,PRODUCCION EST,EST TN COSECHA,DIF EST ENTREGA,%DIF EST ENTREGA,TN EN CAMPO,COMPROMISO REST,DIFERENICA,COMPRADOR,COMPRIM VS EST
0,601,2224,CRUZ VARGAS WILLY WALDO,6261.71,5562.24,0.888294,59.697345,2.705370,56.991975,NaN,...,60.197553,3593.634071,162.856660,5399.383340,0.029279,3430.777411,699.47,2731.307411,1,-2668.075929
1,544,5334,HUALLPA AUCACHI JUSTINO,2958.93,1942.32,0.656426,37.172376,10.711097,26.461279,NaN,...,55.095426,2048.027883,590.132468,1352.187532,0.303829,1457.895415,1016.61,441.285415,1,-910.902117
2,612,2650,CLAROS VASQUEZ JHONNY MAURICIO,2060.35,1854.88,0.900274,33.231943,15.457094,17.774849,NaN,...,47.817524,1589.069238,739.119980,1115.760020,0.398473,849.949258,205.47,644.479258,1,-471.280762


In [7]:
fecha_inf = '27 de agosto de 2025'
fecha_form = '27-08-2025'

fecha_img = ' de agosto de 2025'

cod_grupos = [544]
cod = 544

In [12]:
def crear_reporte(lista_cod_cos):
    for cod in lista_cod_cos:
        grupo = xlsx_entregas[xlsx_entregas['cod_cos'] == cod].copy()
        
        # convertir datos del grupo a objetos
        list_obj_caneros = []
        for index, row in grupo.iterrows():
            canero = {
            'inst' : row['inst'],
            'cod_ca' : row['cod_ca'],
            'nom_ca' : row['nom_ca'],
            'compromiso' : row['TOTAL COMPROMISO'],
            'entrega' : row['TOTAL ENTREGAS'],
            'entrega_porcen' : row['%ENTREGA']
            }
            list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
        # totales de los datos del grupo
        grupo_totales = {
            'compromiso' : grupo['TOTAL COMPROMISO'].sum(),
            'entregas' : grupo['TOTAL ENTREGAS'].sum(),
            'entrega_porcen' : grupo['TOTAL ENTREGAS'].sum() / grupo['TOTAL COMPROMISO'].sum() * 100
        }
        obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
        
        # datos generales de avance de cosecha
        avance = xlsx_avance[xlsx_avance['COD COS'] ==cod]
        avance_totales = {
            'fecha_img' : fecha_img,
            'fecha_informe' : fecha_inf,
            'compromiso' : float(avance['COMPROMISO'].iloc[0]),
            'entrega' : float(avance['ENTREGAS'].iloc[0]),
            'entrega_porcen' : float(avance['%ENTREGA'].iloc[0]) * 100,
            'ha_canha' : float(avance['COSECHADO'].iloc[0] + avance['SIN COSECHAR'].iloc[0]),
            'ha_cosecha' : float(avance['COSECHADO'].iloc[0]),
            'cosecha_porcen' : float(avance['%COSECHA'].iloc[0]) * 100,
            'tch' : float(avance['TCH'].iloc[0])
        }
        # datos generales a objeto
        obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
        
        # generar reporte
        doc = DocxTemplate(RUTA_COMPLETA + '/templates/tpl_rpt_avance_posble_compra.docx')
        context = {
            'insp' : list_obj_caneros,
            'grupo_totales' : obj_grupo_totales,
            'avance' : obj_avance_totales
        }
        doc.render(context)

        #REPROTE POSIBLE COMPRA DE CAÑA
        doc.save(RUTA_COMPLETA + '/informes/' + str(avance['COD CA'].iloc[0]) + '_RPCC_' + fecha_form + '_' + avance['NOMBRE CAÑERO'].iloc[0] + '.docx')
        print('Se generó reporte de grupo:', cod, ' - ', avance['NOMBRE CAÑERO'].iloc[0])

In [9]:
compradores = xlsx_avance[xlsx_avance['COMPRADOR'] == 1]
compradores.head(3)

,COD COS,COD CA,NOMBRE CAÑERO,COMPROMISO,ENTREGAS,%ENTREGA,AREA,COSECHADO,SIN COSECHAR,SEMILLA,...,TCH EST,PRODUCCION EST,EST TN COSECHA,DIF EST ENTREGA,%DIF EST ENTREGA,TN EN CAMPO,COMPROMISO REST,DIFERENICA,COMPRADOR,COMPRIM VS EST
0,601,2224,CRUZ VARGAS WILLY WALDO,6261.71,5562.24,0.888294,59.697345,2.705370,56.991975,NaN,...,60.197553,3593.634071,162.856660,5399.383340,0.029279,3430.777411,699.47,2731.307411,1,-2668.075929
1,544,5334,HUALLPA AUCACHI JUSTINO,2958.93,1942.32,0.656426,37.172376,10.711097,26.461279,NaN,...,55.095426,2048.027883,590.132468,1352.187532,0.303829,1457.895415,1016.61,441.285415,1,-910.902117
2,612,2650,CLAROS VASQUEZ JHONNY MAURICIO,2060.35,1854.88,0.900274,33.231943,15.457094,17.774849,NaN,...,47.817524,1589.069238,739.119980,1115.760020,0.398473,849.949258,205.47,644.479258,1,-471.280762


In [10]:
lista_cods = list(set(compradores['COD COS']))
print(lista_cods)

[258, 264, 399, 401, 19, 544, 545, 424, 307, 564, 311, 183, 58, 446, 576, 193, 452, 459, 463, 208, 343, 601, 602, 604, 609, 612, 613, 614, 109, 249, 508]


In [13]:
crear_reporte(lista_cods)

Se generó reporte de grupo: 258  -  CALVI PADILLA SAVINO
Se generó reporte de grupo: 264  -  CONDORI PADILLA RICHARD
Se generó reporte de grupo: 399  -  POSTIGO VASQUEZ VICTOR HUGO
Se generó reporte de grupo: 401  -  QUISPE ZURITA DELFIN
Se generó reporte de grupo: 19  -  HEVIA Y VACA PEREZ BERNABE
Se generó reporte de grupo: 544  -  HUALLPA AUCACHI JUSTINO
Se generó reporte de grupo: 545  -  RODRIGUEZ FLORA TOMICHA DE
Se generó reporte de grupo: 424  -  CARO SONIA GENOVEVA CLAROS DE
Se generó reporte de grupo: 307  -  GARCIA INES TENORIO DE
Se generó reporte de grupo: 564  -  ALVAREZ ALAVE JORGE
Se generó reporte de grupo: 311  -  MORATO LUIZAGA ALFREDO
Se generó reporte de grupo: 183  -  FLORES CHAVARRIA VICENTE
Se generó reporte de grupo: 58  -  CALLE MERCADO PERCI EINAR
Se generó reporte de grupo: 446  -  RODRIGUEZ VASQUEZ TOMASA
Se generó reporte de grupo: 576  -  JANCO GARCIA JUSTO
Se generó reporte de grupo: 193  -  GARCIA LEON DIOGENES
Se generó reporte de grupo: 452  -  CORDOV

In [37]:
grupo = xlsx_entregas[xlsx_entregas['cod_cos'] == cod].copy()
grupo

,inst,cod_cos,cod_ca,nom_ca,TOTAL COMPROMISO,TOTAL ENTREGAS,%ENTREGA
616,63,544,5277,HUALLPA MENDEZ ADHEMAR,443.91,196.74,44.319795
617,63,544,5334,HUALLPA AUCACHI JUSTINO,864.39,625.99,72.419857
618,63,544,5423,HUALLPA MENDEZ ROLY,215.03,NaN,NaN
619,63,544,5460,HUALLPA MENDEZ GLADYS,893.56,885.39,99.085680
620,63,544,42375,HUALLPA MENDEZ DIEGO,542.04,234.20,43.207143


In [39]:
# convertir datos del grupo a objetos
list_obj_caneros = []
for index, row in grupo.iterrows():
    canero = {
        'inst' : row['inst'],
        'cod_ca' : row['cod_ca'],
        'nom_ca' : row['nom_ca'],
        'compromiso' : row['TOTAL COMPROMISO'],
        'entrega' : row['TOTAL ENTREGAS'],
        'entrega_porcen' : row['%ENTREGA']
    }
    list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
list_obj_caneros

[canero(inst=63, cod_ca=5277, nom_ca='HUALLPA MENDEZ ADHEMAR', compromiso=443.91, entrega=196.74, entrega_porcen=44.31979455294992),
 canero(inst=63, cod_ca=5334, nom_ca='HUALLPA AUCACHI JUSTINO', compromiso=864.39, entrega=625.99, entrega_porcen=72.41985677761195),
 canero(inst=63, cod_ca=5423, nom_ca='HUALLPA MENDEZ ROLY', compromiso=215.03, entrega=nan, entrega_porcen=nan),
 canero(inst=63, cod_ca=5460, nom_ca='HUALLPA MENDEZ GLADYS', compromiso=893.56, entrega=885.39, entrega_porcen=99.08567975289853),
 canero(inst=63, cod_ca=42375, nom_ca='HUALLPA MENDEZ DIEGO', compromiso=542.04, entrega=234.2, entrega_porcen=43.207143384252085)]

In [41]:
# totales de los datos del grupo
grupo_totales = {
    'compromiso' : grupo['TOTAL COMPROMISO'].sum(),
    'entregas' : grupo['TOTAL ENTREGAS'].sum(),
    'entrega_porcen' : grupo['TOTAL ENTREGAS'].sum() / grupo['TOTAL COMPROMISO'].sum() * 100
}
obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
obj_grupo_totales

grupo_totales(compromiso=2958.93, entregas=1942.32, entrega_porcen=65.64264784905355)

In [46]:
# datos generales de avance de cosecha
avance = xlsx_avance[xlsx_avance['COD COS'] ==cod]
avance_totales = {
    'fecha_img' : fecha_img,
    'fecha_informe' : fecha_inf,
    'compromiso' : float(avance['COMPROMISO'].iloc[0]),
    'entrega' : float(avance['ENTREGAS'].iloc[0]),
    'entrega_porcen' : float(avance['%ENTREGA'].iloc[0]) * 100,
    'ha_canha' : float(avance['COSECHADO'].iloc[0] + avance['SIN COSECHAR'].iloc[0]),
    'ha_cosecha' : float(avance['COSECHADO'].iloc[0]),
    'cosecha_porcen' : float(avance['%COSECHA'].iloc[0]) * 100,
    'tch' : float(avance['TCH'].iloc[0])
}
# datos generales a objeto
obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
obj_avance_totales

avance_totales(fecha_img=' de agosto de 2025', fecha_informe='27 de agosto de 2025', compromiso=2958.93, entrega=1942.32, entrega_porcen=65.64264784905355, ha_canha=37.17237600000001, ha_cosecha=10.711097322463035, cosecha_porcen=28.814669588145325, tch=181.33716289988487)

In [52]:
# generar reporte
doc = DocxTemplate(RUTA_COMPLETA + '/templates/tpl_rpt_avance_posble_compra.docx')
context = {
    'insp' : list_obj_caneros,
    'grupo_totales' : obj_grupo_totales,
    'avance' : obj_avance_totales
}
doc.render(context)

#REPROTE POSIBLE COMPRA DE CAÑA
doc.save(RUTA_COMPLETA + '/informes/_' + str(avance['COD CA'].iloc[0]) + '_RPCC_' + fecha_form + '_' + avance['NOMBRE CAÑERO'].iloc[0] + '.docx')
print('Se generó reporte de grupo:', cod)

Se generó reporte de grupo: 544
